<a href="https://colab.research.google.com/github/anupsk5053/SCM256/blob/main/02_quantile_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantile Regression and Inventory Control

In previous sessions, we have learned how to extract information on seasonality and trend from time series data and how to use to construct time-based features for linear regression. We also learned how to use lagged observations as features and how to compare different forecasting models with one another using a variant of k-fold cross validation for time series data.

In this notebook, we will move away from point forecasts and show how we can create probabilistic forecasts based on quantiles, which can be used to create prediction intervals.

We will also see how probabilistic forecasts can be used directly inform to inventory control decisions without first calculating a prediction and then using the error distribution to calculate safety stock levels.

## Dataset: Ice Cream Consumption

The dataset that will serve us as a guiding example is on U.S. ice cream consumption and has been taken from [Kaggle](https://www.kaggle.com/datasets/ronaldocr9803/icecream-consumption-usa). The dataset tracdks monthly demand over several decades and shall be representative of a time series of prduct demand. Let us import the dataset using Pandas.   

In [ ]:
import pandas as pd
df = pd.read_csv("https://lscm.s3.amazonaws.com/machine_learning/ice_cream_consumption.csv", parse_dates=["DATE"], dayfirst=True)
df = df.rename({"DATE":"date","value":"demand"}, axis=1)
df

,date,demand
0,1972-01-01,59.9622
1,1972-02-01,67.0605
2,1972-03-01,74.2350
3,1972-04-01,78.1120
4,1972-05-01,84.7636
...,...,...
571,2019-08-01,102.6085
572,2019-09-01,100.1741
573,2019-10-01,90.1684
574,2019-11-01,79.7223


Now let us plot the demand time series using Plotly Express. The month is shown by hovering over the data point.

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'
px.line(df, x="date", y="demand", hover_name=df.date.dt.month_name())

## Autoregressive Demand Forecast


A simple yet powerful concept in time series analysis is to model the dynamics of a data process purely through past realizations. The rationale behind this is that observations are often correlated in time, so that knowing past observations also holds information about the future. Since correlation in time is often referred to as autocorrelation, the class of regression models that exploits this feature is referred to as autoregressive (AR) models.

While we can technically add any past observation as a feature (as long as there is sufficient data), let us focus on those observations that preceed the current observation by one seasonal cycle, which will give us a so-called seasonal autoregressive (SAR) model.

Including lagged observations as features in Pandas can be cumbersome, because the dimensionality of the feature matrix must match the dimensionality of the target vector. We can achieve this by removing the first 12 observations from the target vector and using the last 12 observations as our features.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
y = df.demand.iloc[12:]
X = df[["demand"]].iloc[:-12]

Having defined our feature matrix and target vector, we can proceed as usual and fit regression model. In this case, we simply fit a linear model.

In [ ]:
reg = LinearRegression().fit(X,y)
y_hat = reg.predict(X)
mad = mean_absolute_error(y,y_hat)

To get a better intuition about what contributes to the model fit, let us plot observations against predictions, and also calculate the MAD.

In [ ]:
px.line(df.iloc[12:].assign(predict=y_hat), x="date", y=["demand","predict"], title="MAD=%.2f"%mad)

The model shows a very good in-sample fit, but since this can be misleading, let us do an additional cross-validation. For this, we use the first five years on record to predict the subsequent year, and then increase train and test size by one year for each fold. This will give us the accuracy of a forecast done at the end of December for the subsequent calendar year.

In [ ]:
n_obs = len(y)
n_skip = 5
n_steps = 12
n_folds = n_obs//n_steps
y_hat = pd.Series(index=y.index, dtype=float)
mad = 0.
for i in range(n_skip,n_folds):
    train_index = i*n_steps
    test_index = (i+1)*n_steps
    X_train, y_train = X.iloc[:train_index], y.iloc[:train_index]
    X_test, y_test = X.iloc[train_index:test_index], y.iloc[train_index:test_index]
    reg = LinearRegression().fit(X_train,y_train)
    y_pred = reg.predict(X_test)
    y_hat.loc[y_test.index] = y_pred
    mad += mean_absolute_error(y_test,y_pred)/(n_folds-n_skip)

Let us again calculate the MAD and plot the predicted demand against the real demand - only this time we use the out-of-sample predidictions.

In [ ]:
px.line(df.iloc[12:].assign(predict=y_hat), x="date", y=["demand","predict"], title="MAD=%.2f"%mad)

The MAD is significantly higher than the in-sample fit, especially in those years during which the long-term trend shifted. This is typical of autoregressive models that are always trailing behind the trend. Nonetheless, the seasonal autoregression is capable of capturing the seasonal pattern quite well.

But how do we now use this forecast to plan our operations? How much capacity do we need? How much to put on inventory? Let us look how we can answer these questions in a data-driven way.

## Inventory Control

In inventory control, we are often interested in finding the optimal basestock level, meaning the optimal quantity to hold on inventory at the beginning of a re-order cycle. The optimal basestock finds the right balance between facing stock-outs when customer demand exceeds on-hand inventory and having excess stocks when demand is too low.

Consider a small super market or retailer that receives deliveries from a central warehouse every morning but who has to order one day in advance. For the sake of simplicy let us assume that there are no fixed re-order cost and long lead times, but that a truck arrives every morning to replenish all items to its basestock level.

### Model Formulation

Let us begin by formulating the decision problem as a simple linear programming problem.

#### Parameters
- $d_t$ demand at time $t$
- $p$ sales price, $c$ purchase cost, $h$ inventory holding cost

#### Decision Variables
- $v_t$ lost sales in $t$
- $u_t$ inventory in $t$
- $s_t$ order-up-to level (basestock) in $t$

#### Static Basestock

The objective of the company is to minimize the total overage and underage cost by choosing the optimal basestock level $s_t$.

$$\begin{align}
\min & \sum_{t=1}^n \left((p-c)v_t+h u_t \right) &\\
\text{s.t. } & u_t - v_t = s_t-d_t, & t=2,...,n \\
& s_t = s_t-1, & t=1,...,n \\
&u_t,v_t,s_t \geq 0, & t=1,...,n
\end{align}
$$

In this model, we assume that demand is known. This is not a problem, as long as we choose basestock levels in a way such that these decisions are made independent of the demand realization in any given period. We do so by forcing basestock levels to take on the same values in every given month, which gives us a static basestock level.

#### Dynamic Basestock

In many real-world settings, we might want to increase the basestock when forecasted demand is higher and decrease the basestock when demand is lower. Think of people buying more icecream during the summer than in winter, or considering the effect of promotions on sales.

Instead of fixing the basestock to a single variable, we will now define the basestock level as a linear function of a set of features, similar to what we do in linear regression.

$$\begin{align}
\min & \sum_{t=1}^n \left((p-c)v_t+h u_t \right) &\\
\text{s.t. } & u_t - v_t = s_t-d_t, & t=1,...,n \\
& s_t = w^\top x_t,\ t=1,...,n\\
&u_t,v_t,s_t \geq 0, & t=1,...,n
\end{align}
$$

The set of features can now include information on seasonal patters, weather forecast, and promotions, much like in time series regression.

#### Linear Quantile Regression

In many cases, inventory holding cost are not available, but companies define a service level that reflects the trade-off between underage and overage cost. Instead of using $p-c$ and $h$ as model parameters, we can also define the objective function by considering the ratio between underage and overage cost,

$$\alpha = \frac{p-c}{p-c+h}.$$

The parameter $\alpha$ can be interpreted as the fillrate, or, $\alpha$-service level. Using this notion of $\alpha$, we obtain the following model

$$\begin{align}
\min & \sum_{t=1}^n \left(\alpha v_t+ (1-\alpha) u_t \right) &\\
\text{s.t. } & u_t - v_t = s_t-d_t, & t=1,...,n \\
& s_t = w^\top x_t,\ t=1,...,n\\
&i_t,b_t,s_t \geq 0, & t=1,...,n
\end{align}
$$

If we relax the requirement that the basestock must always be positive, the above problem is identical to a linear quantile regression problem, where the $\alpha$-quantile is equivalent to the basestock level and where inventory $u_t$ and lost sales $v_t$, express the errors of demand being below or above the selected quantile.

The quantile regression problem if formulated as a generic linear programming problem then looks as follows.

$$\begin{align}
\min_{b,w \in \mathbb{R}; u,v \geq 0} & \frac{1}{n} \sum_{i=1}^n (\alpha u_i + (1-\alpha) v_i) \\
\text{s.t. } & b+\sum_{j=1}^p w_j x_{ij} + u_i - v_i = y_i\ \forall \ i=1,\dots,n
\end{align}$$

#### Pinball Loss

The objective function is also sometimes referred to as pinball loss function due to its asymmetric shape reminding of the two levers in a pinball table.

#### Implementation

Let us now implement the quantile regression problem using the linear programming library FICO Xpress. Below you will find a class `QuantileRegressor` that you can use to fit a linear quantile regression model efficiently.



In [ ]:
!pip install xpress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 6.5 MB/s eta 0:00:00


In [ ]:
import xpress as xp

class QuantileRegressor():

    def __init__(self, X, y):
        import xpress as xp
        size, dim = X.shape
        b = xp.var(lb=-np.inf,ub=np.inf)
        w = np.array([xp.var(lb=-np.inf,ub=np.inf) for i in range(dim)])
        u = np.array([xp.var() for i in range(size)])
        v = np.array([xp.var() for i in range(size)])
        p = xp.problem()
        p.setControl("outputlog", 0)
        p.setControl("dualthreads", 1)
        p.addVariable(b,w,u,v)
        p.addConstraint([b+xp.Sum(X[i]*w)+u[i]-v[i] == y[i] for i in range(size)])
        self._u, self._v, self._b, self._w, self._p, self._size, self._dim, = u, v, b, w, p, size, dim

    def fit(self, alpha=0.5):
        import xpress as xp
        self._p.setObjective(xp.Sum(alpha*self._u[i] for i in range(self._size))+xp.Sum((1-alpha)*self._v[i] for i in range(self._size)))
        self._p.lpoptimize("d")
        self._bsol = self._p.getSolution(self._b)
        self._wsol = np.array([self._p.getSolution(self._w[i]) for i in range(self._dim)])
        self._alpha = alpha
        return self

    def error(y, y_pred, alpha):
        u = np.maximum(y - y_pred,0)
        v = np.maximum(y_pred - y,0)
        return (alpha*u.sum()+(1-alpha)*v.sum())/len(y_pred)

    def predict(self, X):
        return np.dot(X,self._wsol) + self._bsol

Unlike the `BaseEstimator` class from scikit-learn, the feature matrix `X` and the target vector `y` need to be provided with the constructor, whereas the quantile `alpha` needs to be provided with each call to the function `fit`. The reason for this is that the linear programming solver can be warm-started for repeated calls to `fit`.

Let us create a new DataFrame that contains predictions of the 0.05 and 0.95 quantiles as well as the median. To do so, we first create an empty DataFrame, where each column contains the predicted quantile.

In [ ]:
quantiles = [0.05,0.5,0.95]
y_hat = df[["date"]][12:].copy()

Then, we create the quantile regressor object, and for each alpha in our list of quantiles, we fit the corresponding model, store the predicted quantiles, and calculate the total pinball loss. For now, let us skip the hassle of cross-validation and simply take the in-sample fit at face value.

In [ ]:
import numpy as np
qr = QuantileRegressor(X.values,y.values)
total_error = 0.
for alpha in quantiles:
    y_pred = qr.fit(alpha).predict(X)
    total_error += QuantileRegressor.error(y,y_pred,alpha)
    y_hat[alpha] = y_pred
print(total_error)
y_hat

Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/usr/local/lib/python3.10/dist-packages/xpress/license/community-xpauth.xpr')
5.290638826162663


,date,0.05,0.5,0.95
12,1973-01-01,58.986559,63.345630,66.671468
13,1973-02-01,64.397546,69.926696,75.349994
14,1973-03-01,69.866621,76.578410,84.121683
15,1973-04-01,72.822032,80.172904,88.861782
16,1973-05-01,77.892503,86.339820,96.994164
...,...,...,...,...
571,2019-08-01,99.171392,112.220082,131.122754
572,2019-09-01,92.031675,103.536464,119.671570
573,2019-10-01,83.509463,93.171398,106.003041
574,2019-11-01,76.117198,84.180621,94.146803


Finally, let us plot the actual realization along with the inter-quantile range which is the interval into which the realization will be fall with a 90% probability.

In [ ]:
import plotly.graph_objs as go
colors = px.colors.qualitative.Plotly
fig = go.Figure()
new_col = colors[0]
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.05], line=dict(color=new_col, width=0),showlegend=False, mode='lines'))
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.95], fill='tonexty', line=dict(color=new_col, width=0),showlegend=False, mode='lines'))
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.5], line=dict(color=new_col, width=1.5), mode='lines',name="predicted"))
new_col = colors[1]
fig.add_traces(go.Scatter(x=y_hat.date, y=y, line=dict(color=new_col, width=1.5), mode='lines',name="real"))
fig.show()

Unlike the point forecasts obtained using linear regression, the quantile regression provides us with a probabilistic forecasts that tells us in which range it expects an outcome to be and with which probability. This information, in turn, can be used to inform basestock decisions for inventory control. The 95%-quantile for example gives us the optimal basestock level at the 95% $\alpha$-service level.

## Boosting Quantile Regression

Although the relationship between inventory control and quantile regression is easier to understand with linear programming, there is no reason to model the quantile regression problem as a linear model.

Better results than with a linear model may be possible with gradient boosting, which is currently one of the most successfuly machine learning methods for small to medium-sized data sets.

While scikit-learn contains an implementation of gradient boosting, we are going to use LightGBM which is both faster and more accurate. The API of LightGBM resembles the one from scikit-learn so that we can use boosting in the same way as any other estimator of scikit-learn.

Conveniently, LightGBM supports quantiles as loss function out-of-the-box.

In [ ]:
from lightgbm import LGBMRegressor
y_hat = df[["date"]][12:].copy()
total_error = 0.
for alpha in quantiles:
    lgb = LGBMRegressor(alpha=alpha, objective="quantile", metric="quantile")
    model = lgb.fit(X, y)
    y_pred = model.predict(X)
    total_error += QuantileRegressor.error(y,y_pred,alpha)
    y_hat[alpha] = y_pred
print(total_error)

4.60530016188307


Again, let us produce a plot to see what happend when we use this technique.

In [ ]:
import plotly.graph_objs as go
colors = px.colors.qualitative.Plotly
fig = go.Figure()
new_col = colors[0]
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.05], line=dict(color=new_col, width=0),showlegend=False, mode='lines'))
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.95], fill='tonexty', line=dict(color=new_col, width=0),showlegend=False, mode='lines'))
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.5], line=dict(color=new_col, width=1.5), mode='lines',name="predicted"))
new_col = colors[1]
fig.add_traces(go.Scatter(x=y_hat.date, y=y, line=dict(color=new_col, width=1.5), mode='lines',name="real"))
fig.show()

As expected, gradient boosting achieves a better in-sample fit the linear model, since the boosting model has more parameters and is thus more complex. However, keep in mind that we are comparing in-sample fit and have not yet cross-validated the predictors.

## Deep Quantile Regression

An important strength of modern neural networks is their flexibility when building machine learning models. It is this flexibility that enables data scientists to tailor these networks for arbitrary purposes in supervised, unsupervised, and reinforcement learning. The flexibility mainly stems from its ability calulate gradients for arbitrary types of activation and loss functions using [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation). In this notebook, we will exploit this feature to build a neural network for quantile regression.

The loss function for a given $\alpha$-quantile is the asymmetric version of the mean absolute error, in the sense that positive and negative errors are given different weights. The loss function of a neural network with $K$ quantiles as output neurons is given by

$$L(w;\alpha_1,...,\alpha_K) = \sum_{k=1}^K \frac{1}{n} \sum_{i=1}^n \max\left(\alpha_k \sum_{j=1}^p (y_i-w_j x_{ij}),(1-\alpha_k) \sum_{j=1}^p (w_j x_{ij}-y_i) \right)$$

To be able to use this function in a neural network, we will have define our own loss function. The function will receive a batch of real and trained outputs, whereby each output neuron represents a predicted quantile. The loss function for quantiles, $\alpha = (0.05,0.5,0.95)$ using tensorflow's maximum function looks as follows:

In [ ]:
import tensorflow as tf
quantiles = [0.05,0.5,0.95]
def quantile_loss(y_true, y_pred):
    loss = 0.
    for i in range(len(quantiles)):
        err = y_true - y_pred[:,i]
        q = quantiles[i]
        loss += tf.maximum(q*err,-(1-q)*err)
    return loss

### Layers of the Neural Network

We now create a simple neural network that only has one output layer with as many neurons as there are quantiles, one hidden layer with the same number of neurons and ReLU activation function, as well as an input layer.

In [ ]:
from tensorflow import keras
net = keras.models.Sequential()
net.add(keras.layers.Dense(len(quantiles), activation="relu")) #hidden layer
net.add(keras.layers.Dense(len(quantiles))) #output layer

### Model Training

Then we put everything together by compiling the network with our custom loss function and training the model for 100 epochs.

In [ ]:
net.compile(
    loss=quantile_loss
)
net.fit(X, y, epochs=100, verbose=False)
y_hat = pd.DataFrame(data=net.predict(X), index=df.date[12:], columns=quantiles).reset_index()
total_error = 0.
for alpha in quantiles:
    y_pred = y_hat[alpha]
    total_error += QuantileRegressor.error(y,y_pred,alpha)
print(total_error)

18/18 [==============================] - 0s 3ms/step
9.004830732075039


The average pinball loss is significantly higher than for the boosting regressor when using a neural network, which results in quantiles with a much wider range.

In [ ]:
import plotly.graph_objs as go
colors = px.colors.qualitative.Plotly
fig = go.Figure()
new_col = colors[0]
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.05], line=dict(color=new_col, width=0),showlegend=False, mode='lines'))
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.95], fill='tonexty', line=dict(color=new_col, width=0),showlegend=False, mode='lines'))
fig.add_traces(go.Scatter(x=y_hat.date, y=y_hat[0.5], line=dict(color=new_col, width=1.5), mode='lines',name="predicted"))
new_col = colors[1]
fig.add_traces(go.Scatter(x=y_hat.date, y=y, line=dict(color=new_col, width=1.5), mode='lines',name="real"))
fig.show()

Clearly, the linear model and gradient boosting outperform the neural network in-sample. However, keep in mind that we did not calculate the pinball loss out-of sample. Also the quality of the neural network can still be improved by proper parameter tuning. Figuring out the true performance of each model as well as alternative models shall be left to an exercise.